################################################################################
#Licensed Materials - Property of IBM
#(C) Copyright IBM Corp. 2020
#US Government Users Restricted Rights - Use, duplication disclosure restricted
#by GSA ADP Schedule Contract with IBM Corp.
################################################################################

The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below.
Specifically, the Source Components and Sample Materials clause included in the License Information document for
Watson Studio Auto-generated Notebook applies to the auto-generated notebooks. 
By downloading, copying, accessing, or otherwise using the materials, you agree to the License Terms.
http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1

### IBM AutoAI Auto-Generated Notebook v1.13.1

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered,   
there is no guarantee it will successfully execute. This pipeline is optimized for the original dataset.  
The pipeline may fail or produce sub-optimium results if used with different data. For different data,  
please consider returning to AutoAI Experiments to generate a new pipeline. Please read our documentation   
for more information:   
(Cloud Platform) https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .
(Cloud Pak For Data) https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/autoai-notebook.html .


Before modifying the pipeline or trying to re-fit the pipeline, consider:   
The notebook converts dataframes to numpy arrays before fitting the pipeline   
(a current restriction of the preprocessor pipeline). The known_values_list is passed by reference   
and populated with categorical values during fit of the preprocessing pipeline. Delete its members before re-fitting.

### Representing Pipeline_2 


### 1. Set Up
If lightgbm or xgboost installation fails, please follow:
 - [lightgbm docs](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html)
 - [xgboost docs](https://xgboost.readthedocs.io/en/latest/build.html)

In [2]:

try:
    import autoai_libs
except Exception as e:
    import subprocess
    out = subprocess.check_output('pip install autoai-libs'.split(' '))
    for line in out.splitlines():
        print(line)
    import autoai_libs
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import inf, nan, dtype, mean
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]

lightgbm, if needed, will be installed and imported later


In [3]:
# compose a decorator to assist pipeline instantiation via import of modules and installation of packages
def decorator_retries(func):
    def install_import_retry(*args, **kwargs):
        retries = 0
        successful = False
        failed_retries = 0
        while retries < 100 and failed_retries < 10 and not successful:
            retries += 1
            failed_retries += 1
            try:
                result = func(*args, **kwargs)
                successful = True
            except Exception as e:
                estr = str(e)
                if estr.startswith('name ') and estr.endswith(' is not defined'):
                    try:
                        import importlib
                        module_name = estr.split("'")[1]
                        module = importlib.import_module(module_name)
                        globals().update({module_name: module})
                        print('import successful for ' + module_name)
                        failed_retries -= 1
                    except Exception as import_failure:
                        print('import of ' + module_name + ' failed with: ' + str(import_failure))
                        import subprocess
                        if module_name == 'lightgbm':
                            try:
                                print('attempting pip install of ' + module_name)
                                process = subprocess.Popen('pip install ' + module_name, shell=True)
                                process.wait()
                            except Exception as E:
                                print(E)
                                try:
                                    import sys
                                    print('attempting conda install of ' + module_name)
                                    process = subprocess.Popen('conda install --yes --prefix {sys.prefix} -c powerai ' + module_name, shell = True)
                                    process.wait()
                                except Exception as lightgbm_installation_error:
                                    print('lightgbm installation failed!' + lightgbm_installation_error)
                        else:
                            print('attempting pip install of ' + module_name)
                            process = subprocess.Popen('pip install ' + module_name, shell=True)
                            process.wait()
                        try:
                            print('re-attempting import of ' + module_name)
                            module = importlib.import_module(module_name)
                            globals().update({module_name: module})
                            print('import successful for ' + module_name)
                            failed_retries -= 1
                        except Exception as import_or_installation_failure:
                            print('failure installing and/or importing ' + module_name + ' error was: ' + str(
                                import_or_installation_failure))
                            raise (ModuleNotFoundError('Missing package in environment for ' + module_name +
                                                       '? Try import and/or pip install manually?'))
                elif type(e) is AttributeError:
                    if 'module ' in estr and ' has no attribute ' in estr:
                        pieces = estr.split("'")
                        if len(pieces) == 5:
                            try:
                                import importlib
                                print('re-attempting import of ' + pieces[3] + ' from ' + pieces[1])
                                module = importlib.import_module('.' + pieces[3], pieces[1])
                                failed_retries -= 1
                            except:
                                print('failed attempt to import ' + pieces[3])
                                raise (e)
                        else:
                            raise (e)
                else:
                    raise (e)
        if successful:
            print('Pipeline successfully instantiated')
        else:
            raise (ModuleNotFoundError(
                'Remaining missing imports/packages in environment? Retry cell and/or try pip install manually?'))
        return result
    return install_import_retry


### 2. Compose Pipeline

In [4]:
# metadata necessary to replicate AutoAI scores with the pipeline
_input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'term_deposit_subscribed', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 1, 'pn': 'P2', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'f1', 'random_state': 33, 'data_source': ''}

# define a function to compose the pipeline, and invoke it
@decorator_retries
def compose_pipeline():
    import numpy
    from numpy import nan, dtype, mean
    #
    # composing steps for toplevel Pipeline
    #
    _input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'term_deposit_subscribed', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 1, 'pn': 'P2', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'f1', 'random_state': 33, 'data_source': ''}
    steps = []
    steps.append(('column_selector', autoai_libs.transformers.exportable.ColumnSelector(activate_flag=True, columns_indices_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])))
    #
    # composing steps for preprocessor Pipeline
    #
    preprocessor__input_metadata = None
    preprocessor_steps = []
    #
    # composing steps for preprocessor_features FeatureUnion
    #
    preprocessor_features_transformer_list = []
    #
    # composing steps for preprocessor_features_categorical Pipeline
    #
    preprocessor_features_categorical__input_metadata = None
    preprocessor_features_categorical_steps = []
    preprocessor_features_categorical_steps.append(('cat_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 14, 15])))
    preprocessor_features_categorical_steps.append(('cat_compress_strings', autoai_libs.transformers.exportable.CompressStrings(activate_flag=True, compress_type='hash', dtypes_list=['float_int_num', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'int_num', 'char_str', 'float_int_num', 'int_num', 'char_str'], missing_values_reference_list=['', '-', '?', nan], misslist_list=[[nan], [], [nan], [], [], [], [nan], [], [], [], [nan], [], []])))
    preprocessor_features_categorical_steps.append(('cat_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[nan])))
    preprocessor_features_categorical_steps.append(('cat_unknown_replacer', autoai_libs.transformers.exportable.NumpyReplaceUnknownValues(filling_values=nan, filling_values_list=[100001, nan, nan, nan, nan, nan, nan, nan, nan, nan, 100001, nan, nan], known_values_list=[[18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 91.0, 93.0, 95.0, 97.0], [299961598826231006696258074368434668367, 118306684911806858809743754946421076486, 98478044226004807457109703722838050713, 159321454560324332308049628502064282166, 277865529956089617403127183248900841086, 219787300200495054367126911617512439045, 151187835775071385968551753808908784991, 22333232251373414705922969281349238736, 273090479048997458791671709926862556104, 313124412425195552816929405018604615431, 283364312271660996400883763491949419861, 230715114430321724850934713783197932106], [194429803966179465246868280202713319745, 94794063833506540693839539212819851226, 294237220628406284433086281919417668540], [74996123215559660479243836105077404498, 41848077678477651631734462707804428744, 257113281635338527704033019287091017778, 230715114430321724850934713783197932106], [169662019754859674907370307324476606919, 220736790854050750400968561922076059550], [169662019754859674907370307324476606919, 220736790854050750400968561922076059550], [169662019754859674907370307324476606919, 220736790854050750400968561922076059550], [319957305852170225521112812168991375559, 246880704061628641367654320647468079300, 230715114430321724850934713783197932106], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], [275784346690187514303816552994600456818, 244438906729132108043330712747501799918, 42445127782472950554797800024661542043, 286741330029533493643110530981198886719, 332514351007991731146988251875597097923, 319495134707690236228278781874251052379, 142685692963625560484523432217459905823, 127158603276756608243638260060214092276, 205092795760493320362203874509052033707, 237227646578375964385177369782869262798, 38360030800745582009956586968796154758, 65540376959099362095454126872440526870], [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 41.0, 43.0, 44.0, 46.0, 55.0, 58.0, 63.0], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 35, 37, 38, 40, 41, 51, 55, 58, 275], [83588513090993268840740664170047631393, 161330870025401753341235738137984802479, 50576416681569067733749004571610022006, 230715114430321724850934713783197932106]], missing_values_reference_list=['', '-', '?', nan])))
    preprocessor_features_categorical_steps.append(('boolean2float_transformer', autoai_libs.transformers.exportable.boolean2float(activate_flag=True)))
    preprocessor_features_categorical_steps.append(('cat_imputer', autoai_libs.transformers.exportable.CatImputer(activate_flag=True, missing_values=nan, sklearn_version_family='20', strategy='most_frequent')))
    preprocessor_features_categorical_steps.append(('cat_encoder', autoai_libs.transformers.exportable.CatEncoder(activate_flag=True, categories='auto', dtype=numpy.float64, encoding='ordinal', handle_unknown='error', sklearn_version_family='20')))
    preprocessor_features_categorical_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_categorical_ Pipeline
    preprocessor_features_categorical_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_categorical_steps)
    preprocessor_features_transformer_list.append(('categorical', preprocessor_features_categorical_pipeline))
    #
    # composing steps for preprocessor_features_numeric Pipeline
    #
    preprocessor_features_numeric__input_metadata = None
    preprocessor_features_numeric_steps = []
    preprocessor_features_numeric_steps.append(('num_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[5, 11, 13])))
    preprocessor_features_numeric_steps.append(('num_floatstr2float_transformer', autoai_libs.transformers.exportable.FloatStr2Float(activate_flag=True, dtypes_list=['float_int_num', 'float_int_num', 'float_int_num'], missing_values_reference_list=[nan])))
    preprocessor_features_numeric_steps.append(('num_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[nan])))
    preprocessor_features_numeric_steps.append(('num_imputer', autoai_libs.transformers.exportable.NumImputer(activate_flag=True, missing_values=nan, strategy='median')))
    preprocessor_features_numeric_steps.append(('num_scaler', autoai_libs.transformers.exportable.OptStandardScaler(num_scaler_copy=None, num_scaler_with_mean=None, num_scaler_with_std=None, use_scaler_flag=False)))
    preprocessor_features_numeric_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_numeric_ Pipeline
    preprocessor_features_numeric_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_numeric_steps)
    preprocessor_features_transformer_list.append(('numeric', preprocessor_features_numeric_pipeline))
    # assembling preprocessor_features_ FeatureUnion
    preprocessor_features_pipeline = sklearn.pipeline.FeatureUnion(transformer_list=preprocessor_features_transformer_list)
    preprocessor_steps.append(('features', preprocessor_features_pipeline))
    preprocessor_steps.append(('permuter', autoai_libs.transformers.exportable.NumpyPermuteArray(axis=0, permutation_indices=[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 14, 15, 5, 11, 13])))
    # assembling preprocessor_ Pipeline
    preprocessor_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_steps)
    steps.append(('preprocessor', preprocessor_pipeline))
    steps.append(('estimator', xgboost.sklearn.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1, gamma=0.671100517258099, learning_rate=0.02030558503862224, max_delta_step=0, max_depth=8, min_child_weight=13, missing=None, n_estimators=658, n_jobs=2, nthread=None, objective='binary:logistic', random_state=33, reg_alpha=0.9995596900777052, reg_lambda=0.18785548958641501, scale_pos_weight=2.8852029379107282, seed=None, silent=True, subsample=0.6416021899604141, verbosity=0, tree_method='hist')))
    # assembling  Pipeline
    pipeline = sklearn.pipeline.Pipeline(steps=steps)
    return pipeline
pipeline = compose_pipeline()


Pipeline successfully instantiated


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


### 3. Extract needed parameter values from AutoAI run metadata

In [5]:

# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
#data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
if 'data_provenance' in _input_metadata:
    data_provenance = _input_metadata['data_provenance']
else:
    data_provenance = None
if 'pos_label' in _input_metadata and learning_type == 'classification':
    pos_label = _input_metadata['pos_label']
else:
    pos_label = None


### 4. Create dataframe from dataset in Cloud Object Storage

In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
#  Read the data as a dataframe
import pandas as pd

csv_encodings=['UTF-8','Latin-1'] # supplement list of encodings as necessary for your data
df = None
readable = None  # if automatic detection fails, you can supply a filename here

# First, obtain a readable object
# Cloud Object Storage data access
# Assumes COS credentials are in a dictionary named 'credentials_0'
   
credentials = df = globals().get('credentials_0')       
if readable is None and credentials is not None :
    try:
        import types
        import pandas as pd
        import io
    except Exception as import_exception:
        print('Error with importing packages - check if you installed them on your environment')
    try:
        if credentials['SERVICE_NAME'] == 's3':
            try:
                from botocore.client import Config
                import ibm_boto3
            except Exception as import_exception:
                print('Installing required packages!')
                !pip install ibm-cos-sdk
                print('accessing data via Cloud Object Storage')
            try:
                client = ibm_boto3.client(service_name=credentials['SERVICE_NAME'],
                                    ibm_api_key_id=credentials['APIKEY'],
                                    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                                    config=Config(signature_version='oauth'),
                                    endpoint_url=credentials['ENDPOINT'])
            except Exception as cos_exception:
                print('unable to create client for cloud object storage')
            try:
                readable = client.get_object(Bucket=credentials['BUCKET'],Key=credentials['FILE'])['Body']
                # add missing __iter__ method, so pandas accepts readable as file-like object
                if not hasattr(readable, "__iter__"): readable.__iter__ = types.MethodType( __iter__, readable )
            except Exception as cos_access_exception:
                print('unable to access data object in cloud object storage with credentials supplied')
        elif credentials['SERVICE_NAME'] == 'fs':
            print('accessing data via File System')
            try:
                if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                    df = pd.read_excel(credentials['FILE'])
                else:
                    df = pd.read_csv(credentials['FILE'], sep = _input_metadata['separator'])
            except Exception as FS_access_exception:
                print('unable to access data object in File System with path supplied') 
    except Exception as data_access_exception:
        print('unable to access data object with credentials supplied') 

# IBM Cloud Pak for Data data access
project_filename = globals().get('project_filename')       
if readable is None and 'credentials_0' in globals() and 'ASSET_ID' in credentials_0:
    project_filename = credentials_0['ASSET_ID']
if project_filename != None and project_filename != '1':
    print('attempting project_lib access to ' + str(project_filename))
    try:
        from project_lib import Project
        project = Project.access()
        storage_credentials = project.get_storage_metadata()
        readable = project.get_file(project_filename)
    except Exception as project_exception:
        print('unable to access data using the project_lib interface and filename supplied')

# Use data_provenance as filename if other access mechanisms are unsuccessful
if readable is None and type(data_provenance) is str:
    print('attempting to access local file using path and name ' + data_provenance)
    readable = data_provenance

# Second, use pd.read_csv to read object, iterating over list of csv_encodings until successful
if readable is not None:
    for encoding in csv_encodings:
        try:
            if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                buffer = io.BytesIO(readable.read())
                buffer.seek(0)
                df = pd.read_excel(buffer, encoding=encoding,sheet_name=_input_metadata['excel_sheet'])
            else:
                df = pd.read_csv(readable, encoding = encoding, sep = _input_metadata['separator'])
            print('successfully loaded dataframe using encoding = ' + str(encoding))
            break
        except Exception as exception_dataread:
            print('unable to read csv using encoding ' + str(encoding))
            print('handled error was ' + str(exception_dataread))
    if df is None:
        print('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings))
        print(f'Please use \'insert to code\' on data panel to load dataframe.')
        raise(ValueError('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings)))

if isinstance(df,pd.DataFrame):
    print('Data loaded succesfully')
    if _input_metadata.get('subsampling') is not None:
        df = df.sample(frac=_input_metadata['subsampling'], random_state=_input_metadata['random_state']) if _input_metadata['subsampling'] <= 1.0 else df.sample(n=_input_metadata['subsampling'], random_state=_input_metadata['random_state'])
else:
    print('Data cannot be loaded with credentials supplied, please provide DataFrame with training data.')

successfully loaded dataframe using encoding = UTF-8
Data loaded succesfully


### 5. Preprocess Data

In [8]:
# Drop rows whose target is not defined
target = target_label_name # your target name here
if learning_type == 'regression':
    df[target] = pd.to_numeric(df[target], errors='coerce')
df.dropna('rows', how='any', subset=[target], inplace=True)


In [9]:
# extract X and y
df_X = df.drop(columns=[target])
df_y = df[target]


In [10]:
# Detach preprocessing pipeline (which needs to see all training data)
preprocessor_index = -1
preprocessing_steps = [] 
for i, step in enumerate(pipeline.steps):
    preprocessing_steps.append(step)
    if step[0]=='preprocessor':
        preprocessor_index = i
        break
#if len(pipeline.steps) > preprocessor_index+1 and pipeline.steps[preprocessor_index + 1][0] == 'cognito':
    #preprocessor_index += 1
    #preprocessing_steps.append(pipeline.steps[preprocessor_index])
if preprocessor_index >= 0:
    preprocessing_pipeline = Pipeline(memory=pipeline.memory, steps=preprocessing_steps)
    pipeline = Pipeline(steps=pipeline.steps[preprocessor_index+1:])

In [11]:
# Preprocess X
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
known_values_list.clear()  #  known_values_list is filled in by the preprocessing_pipeline if needed
preprocessing_pipeline.fit(df_X.values, df_y.values)
X_prep = preprocessing_pipeline.transform(df_X.values)

### 6. Split data into Training and Holdout sets

In [12]:
# determine learning_type and perform holdout split (stratify conditionally)
if learning_type is None:
    # When the problem type is not available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
    # Caution:  This can mis-classify regression targets that can be expressed as integers as multiclass, in which case manually override the learning_type
    from sklearn.utils.multiclass import type_of_target
    if type_of_target(df_y.values) in ['multiclass', 'binary']:
        learning_type = 'classification'
    else:
        learning_type = 'regression'
    print('learning_type determined by type_of_target as:',learning_type)
else:
    print('learning_type specified as:',learning_type)
    
from sklearn.model_selection import train_test_split
if learning_type == 'classification':
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state)


learning_type specified as: classification


#### 7. Generate features via Feature Engineering pipeline

In [13]:
#Detach Feature Engineering pipeline if next, fit it, and transform the training data
fe_pipeline = None
if pipeline.steps[0][0] == 'cognito':
    try:
        fe_pipeline = Pipeline(steps=[pipeline.steps[0]])
        X = fe_pipeline.fit_transform(X, y)
        X_holdout = fe_pipeline.transform(X_holdout)
        pipeline.steps = pipeline.steps[1:]
    except IndexError:
        try:
            print('Trying to compose pipeline with some of cognito steps')
            fe_pipeline = Pipeline(steps = list([pipeline.steps[0][1].steps[0],pipeline.steps[0][1].steps[1]]))
            X = fe_pipeline.fit_transform(X, y)
            X_holdout = fe_pipeline.transform(X_holdout)
            pipeline.steps = pipeline.steps[1:]
        except IndexError:
            print('Composing pipeline without cognito steps!')
            pipeline.steps = pipeline.steps[1:]


 ### 8. Additional setup: Define a function that returns a scorer for the target's positive label

In [14]:
# create a function to produce a scorer for a given positive label
def make_pos_label_scorer(scorer, pos_label):
    kwargs = {'pos_label':pos_label}
    for prop in ['needs_proba', 'needs_threshold']:
        if prop+'=True' in scorer._factory_args():
            kwargs[prop] = True
    if scorer._sign == -1:
        kwargs['greater_is_better'] = False
    from sklearn.metrics import make_scorer
    scorer=make_scorer(scorer._score_func, **kwargs)
    return scorer

### 9. Fit pipeline, predict on Holdout set, calculate score, perform cross-validation

In [15]:
# fit the remainder of the pipeline on the training data
pipeline.fit(X,y)

[09:40:43] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.


Pipeline(memory=None,
     steps=[('estimator', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.671100517258099,
       learning_rate=0.02030558503862224, max_delta_step=0, max_depth=8,
       min_child_weight=13, missing=None, n_estimators=658, n_job...282, seed=None, silent=True,
       subsample=0.6416021899604141, tree_method='hist', verbosity=0))])

In [16]:
# predict on the holdout data
y_pred = pipeline.predict(X_holdout)

In [17]:
# compute score for the optimization metric
# scorer may need pos_label, but not all scorers take pos_label parameter
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
score = None
#score = scorer(pipeline, X_holdout, y_holdout)  # this would suffice for simple cases
pos_label = None  # if you want to supply the pos_label, specify it here
if pos_label is None and 'pos_label' in _input_metadata:
    pos_label=_input_metadata['pos_label']
try:
    score = scorer(pipeline, X_holdout, y_holdout)
except Exception as e1:
    if learning_type is "classification" and (pos_label is None or str(pos_label)==''):
        print('You may have to provide a value for pos_label in order for a score to be calculated.')
        raise(e1)
    else:
        exception_string=str(e1)
        if 'pos_label' in exception_string:
            try:
                scorer = make_pos_label_scorer(scorer, pos_label=pos_label)
                score = scorer(pipeline, X_holdout, y_holdout)
                print('Retry was successful with pos_label supplied to scorer')
            except Exception as e2:
                print('Initial attempt to use scorer failed.  Exception was:')
                print(e1)
                print('')
                print('Retry with pos_label failed.  Exception was:')
                print(e2)
        else:
            raise(e1)

if score is not None:
    print(score)

0.6777920410783055


In [18]:
# cross_validate pipeline using training data
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold
if learning_type == 'classification':
    fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
else:
    fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
cv_results = cross_validate(pipeline, X, y, cv=fold_generator, scoring={optimization_metric:scorer}, return_train_score=True)
import numpy as np
np.mean(cv_results['test_' + optimization_metric])

[09:40:48] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[09:40:53] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[09:40:57] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.


0.6419591005902086

In [19]:
cv_results

{'fit_time': array([3.73766446, 2.99079132, 2.98098707]),
 'score_time': array([0.39129353, 0.42210841, 0.39973974]),
 'test_f1': array([0.64329393, 0.6349481 , 0.64763528]),
 'train_f1': array([0.78813205, 0.78824764, 0.78625   ])}